Первоначальная версия датасета состоит из десяти столбцов, содержащих следующую информацию:

 - **Restaurant_id** — идентификационный номер ресторана / сети ресторанов;
 - **City** — город, в котором находится ресторан;
 - **Cuisine Style** — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
 - **Ranking** — место, которое занимает данный ресторан среди всех ресторанов своего города;
 - **Rating** — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
 - **Price Range** — диапазон цен в ресторане;
 - **Number of Reviews** — количество отзывов о ресторане;
 - **Reviews** — данные о двух отзывах, которые отображаются на сайте ресторана;
 - **URL_TA** — URL страницы ресторана на TripAdvosor;
 - **ID_TA** — идентификатор ресторана в базе данных TripAdvisor.

Задачу, которая стоит перед вами, можно свести к трём пунктам:

 - Удалить из датафрейма столбцы, данные в которых представлены не числами (это вы уже сделали, и нужно просто повторить знакомые действия, но в этот раз выполнить данный шаг в последнюю очередь).
 - Избавиться от пропущенных (None) значений (на предыдущем шаге мы делали это самым грубым из всех возможных способов; сейчас попробуем подойти к процессу более гибко).
 - Создать новые столбцы с данными, используя для этого информацию, содержащуюся в других столбцах датафрейма (например, можно добавить столбец, сообщающий, сколько дней прошло со дня публикации последнего отзыва, отображённого на сайте).
 
С другой стороны, в этом задании масса подводных камней.

Вот по каким критериям проект будет оценивать ментор:

 - Качество кода (ясность, соответствие стандартам оформления pep-8, наличие комментариев).
 - Наличие визуализации с выводами (корреляция и важность признаков).
 - Feature Engineering и обогащение датасета (4 - наличие редких/уникальных предикторов модели).
 - Работа с метрикой МАЕ.
 - Использование внешних источников данных.

In [109]:
# импорт библиотек
import pandas as pd
import numpy as np
import datetime

In [110]:
# фуекции используемые в ноутбуке



In [111]:
# чтение данных
df = pd.read_csv('./Preproject3_data/main_task.xlt')

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
Restaurant_id        40000 non-null object
City                 40000 non-null object
Cuisine Style        30717 non-null object
Ranking              40000 non-null float64
Rating               40000 non-null float64
Price Range          26114 non-null object
Number of Reviews    37457 non-null float64
Reviews              40000 non-null object
URL_TA               40000 non-null object
ID_TA                40000 non-null object
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [113]:
df.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


In [114]:
df.columns = df.columns.str.replace(' ','_')

### ТРЕНИРОВКА

### 2.1 Задание 1
Какие столбцы НЕ содержат пропущенных (None) значений?

In [115]:
df.columns[df.isna().sum() == 0]

Index(['Restaurant_id', 'City', 'Ranking', 'Rating', 'Reviews', 'URL_TA',
       'ID_TA'],
      dtype='object')

### 2.2 Задание 2
В каких столбцах данные хранятся в числовом формате?


In [116]:
df.columns[df.dtypes != np.object]

Index(['Ranking', 'Rating', 'Number_of_Reviews'], dtype='object')

### 2.3 Задание 3
В каких столбцах хранящиеся данные представляют собой список?

In [117]:
for vol in df.iloc[0]:
    if type(vol) == list: print(vol.index)

### 4.1 Вопросы о ценах
Сколько вариантов непустых значений встречается в столбце Price Range?

In [118]:
df.Price_Range.unique()

array(['$$ - $$$', nan, '$$$$', '$'], dtype=object)

Сколько ресторанов относятся к среднему ценовому сегменту?

In [119]:
df.Price_Range.value_counts()

$$ - $$$    18412
$            6279
$$$$         1423
Name: Price_Range, dtype: int64

### 4.2 Вопрос о городах
Сколько городов представлено в наборе данных?

In [120]:
df.City.nunique()

31

### 4.3 Вопросы о кухнях
Сколько типов кухонь представлено в наборе данных?

In [121]:
cuisine_to_list = lambda x: x.replace('[','').replace("'","").replace(']','').split(', ') 

total_cuisine = df.Cuisine_Style.dropna().apply(cuisine_to_list).sum()
len(set(total_cuisine))

125

Какая кухня представлена в наибольшем количестве ресторанов? Введите название кухни без кавычек или апострофов.

In [122]:
pd.Series(total_cuisine).value_counts().idxmax()

'Vegetarian Friendly'

Какое среднее количество кухонь предлагается в одном ресторане? Если в данных отсутствует информация о типах кухонь, то считайте, что в этом ресторане предлагается только один тип кухни. Ответ округлите до одного знака после запятой.

In [123]:
total_cuisine_nan = df.Cuisine_Style.fillna("['nan']").apply(cuisine_to_list).sum()

In [124]:
round(len(total_cuisine_nan)/len(df.Cuisine_Style),1)

2.6

### 4.4 Вопросы об отзывах
Когда был оставлен самый свежий отзыв? Введите ответ в формате yyyy-mm-dd.

In [125]:
Rev_data_to_list = lambda x: x[2:-2].replace("'","").split('], [')[1].split(', ')
str_to_data = lambda x: datetime.datetime.strptime(x, '%m/%d/%Y').date()

df['Reviews_data'] = df.Reviews.apply(Rev_data_to_list)
df['Reviews_data'] = df.Reviews_data.apply(lambda x: [str_to_data(y) for y in x] if x[0] else np.nan)

print(max(Rev_data_total.dropna().sum()))

2018-02-26


Какое максимальное количество дней отделяет даты публикации отзывов, размещённых на сайте ресторана? Введите количество дней в виде целого числа.

In [126]:
max(Rev_data_total.dropna().apply(lambda x: max(x) - min(x)))

Timedelta('3207 days 00:00:00')